<a href="https://colab.research.google.com/github/SakshamTalwar12/agriculturefinetune/blob/main/agriculturefinetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

In [ ]:
!pip install huggingface_hub

In [ ]:
pip install torch transformers datasets peft trl bitsandbytes

In [ ]:
import torch
from trl import SFTTrainer
from peft import LoraConfig
from datasets import load_dataset
from transformers import (AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline)


In [ ]:
llama_model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path = "soma77/agriculture_advisor",
                                                   quantization_config = BitsAndBytesConfig(load_in_4bit = True,
                                                                                            bnb_4bit_compute_dtype = getattr(torch, "float16"),
                                                                                            bnb_4bit_quant_type = "nf4"),  device_map="auto")


In [ ]:
llama_model.config.use_cache = False
llama_model.config.pretraining_tp = 1

In [ ]:
llama_tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path = "soma77/agriculture_advisor", trust_remote_code = True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.padding_side = "right"

In [ ]:
training_arguments = TrainingArguments(output_dir = "./results", per_device_train_batch_size = 1, max_steps = 100)


In [ ]:
from datasets import load_dataset

dataset = load_dataset("YuvrajSingh9886/Agriculture-Soil-QA-Pairs-Dataset", split="train")
print(dataset.column_names)


In [ ]:
from datasets import load_dataset
dataset = load_dataset("YuvrajSingh9886/Agriculture-Soil-QA-Pairs-Dataset", split="train")
def format_example(example):
    return {
        "text": (
            f"Context: {example['QUESTION.paragraph']}\n"
            f"Question: {example['QUESTION.question']}\n"
            f"Answer: {example['ANSWER']}"
        )
    }

formatted_dataset = dataset.map(format_example, remove_columns=dataset.column_names)


In [ ]:
llama_sft_trainer = SFTTrainer( model=llama_model, args=training_arguments, train_dataset=formatted_dataset, peft_config=LoraConfig( task_type="CAUSAL_LM", r=64, lora_alpha=16, lora_dropout=0.1))


In [ ]:
llama_sft_trainer.train()

In [ ]:
user_prompt = "Please tell me about red soil management"
text_generation_pipeline = pipeline(task = "text-generation", model = llama_model, tokenizer = llama_tokenizer, max_length = 300)
model_answer = text_generation_pipeline(f"{user_prompt}")
print(model_answer[0]['generated_text'])